## **1 кейс**

**Работа с логами**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить необходимый для работы файл.

In [1]:
!wget https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log

--2025-03-19 05:53:56--  https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.github.com (gist.github.com)... 140.82.114.4
Connecting to gist.github.com (gist.github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log [following]
--2025-03-19 05:53:56--  https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 459418 (449K) [text/plain]
Saving to: ‘auto_purchase.log’

auto_purchase.log   100%[===================>] 448.65K  --.-KB/s    in 0.05s   

2025-03-19 05:53:57 (8.90

Чтобы посмотреть как он выглядит выполните следующую ячейку.

In [ ]:
with open('auto_purchase.log', 'r') as f:
    lines = f.readlines()

for line in lines[0-10]:
  print(line)

### **Решения**

#### **Задача 1**

Ваша задача написать функцию `count_success_and_failure`, которая принимает на вход путь к файлу с логами и подсчитывает количество успешных продлений и ошибок при списании. Функция должна вернуть кортеж из двух значений: количества успешных попыток и неуспешных.

**Решение**

Напишите свое решение ниже

In [ ]:
# Судя по логу, в INFO записывается вся информация, даже когда в итоге продление не удалось. Поэтому из этого количества следует вычесть ошибки
def count_success_and_failure(file_path):
  with open(file_path, 'r') as f:
    lines = f.readlines()
    err_count = 0
    pay_count = 0
    for line in lines:
      condition = line.find('Обновляем подписку пользователю id') == -1
      if line.startswith('ERROR') and line.endswith('ошибка при списании:\n'):
        err_count += 1
      if line.startswith('INFO') and not condition:
        pay_count += 1

    return pay_count - err_count, err_count

count_success_and_failure('auto_purchase.log')

(1034, 186)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
res = count_success_and_failure('auto_purchase.log')

try:
    assert res == (1034, 186)
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 2**

Ваша задача написать функцию `auto_renewal_sub`, которая принимает на вход путь к файлу с логами и обрабатывает количество клиентов с автопродлением подписки. Мы хотим посмотреть на изменение этого показателя в динамике: посчитайте сглаженные значения с помощью метода скользящего среднего и метода медианного сглаживания.  

**Примечание:** При сглаживании берем все предыдущие значения, включая текущее, будущие значения не берем. Если в один день наблюдаем несколько записей об автопродлении - берем максимальное из имеющихся число клиентов с подпиской.

Функция должна записать в файл `auto_renewal_sub.txt` два списка, предварив их соответствущими обозначениями:

`Среднее: [2.0, 1.0, 0.67...]`

`Медиана: [2, 2, 0...]`

**Решение**

Напишите свое решение ниже

In [33]:
def calculate_median(data):
  # Сортируем список
  sorted_data = sorted(data)
  n = len(sorted_data)
  # Проверяем, четное или нечетное количество элементов
  if n % 2 == 1:
      median = sorted_data[n // 2]
  else:
      median = int((sorted_data[n // 2 - 1] + sorted_data[n // 2]) / 2)
  return median

def calculate_mean(data):
  return round(sum(data)/len(data),2)


def auto_renewal_sub(log_file_path):
  with open(log_file_path, 'r') as f:
    lines = f.readlines()
    data = {}
    # Проходим по логу и ищем итоговые строки по каждому дню
    for line in lines:
      pos = line.find('Cегодня ')
      subscribe = line.find('подписки:')
      if pos != -1:
        # Для выполнения условия: Если в один день наблюдаем несколько записей
        # об автопродлении - берем максимальное из имеющихся число клиентов с подпиской
        if line[pos+8:pos+18] in data:
          data[line[pos+8:pos+18]] = max(int(data[line[pos+8:pos+18]]), \
                                         int(line[subscribe+10:].split('\n')[0]))
        else:
          data[line[pos+8:pos+18]] = int(line[subscribe+10:].split('\n')[0])
    # На всякий случай отсортируем по датам по возрастанию
    data = {key: data[key] for key in sorted(data)}
    # Теперь в data хранятся пары 'дата': количество подписчиков
    # Для именно этой задачи не сильно пригодится, но так удобнее потом что-то ещё делать

    # Создадим словарь со значениями подписок
    subs = []
    for key, value in data.items():
      subs.append(value)

    mean = [calculate_mean(subs[:i]) for i in range(1,len(subs)+1)]
    median = [calculate_median(subs[:i]) for i in range(1,len(subs)+1)]
    # Запишем результаты в файл, как требуют условия
    with open('auto_renewal_sub.txt', 'w') as f:
      f.write(f'Среднее: {mean}')
      f.write(f'Медиана: {median}')

auto_renewal_sub('auto_purchase.log')

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [34]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt

import pandas as pd

user_answer = pd.read_csv('auto_renewal_sub.txt')
correct_answer = pd.read_csv('cor_auto_renewal.txt')

--2025-03-19 08:09:41--  https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt
Resolving gist.github.com (gist.github.com)... 140.82.112.3
Connecting to gist.github.com (gist.github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt [following]
--2025-03-19 08:09:41--  https://gist.githubusercontent.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2431 (2.4K) [text/plain]
Saving to: ‘cor_auto_renewal.txt.1’

cor_auto_renewal.tx 100%[===================>]   2.37K  --.-KB/s    in 0s      

2025-03-19 08

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!


#### **Задача 3**

Напишите функцию `sub_renewal_by_day`, которая принимает на вход путь к файлу с логами и анализирует взаимосвязь дня продления подписки и количества продлений в этот день. Функция должна записать в файл `weekdays.txt` аналитическую записку в формате:

**`Количество обновлений подписки по дням недели:`**

**`Понедельник: 6`**

**`Вторник: 7`**

**`Среда: 8`**

**`...`**

**Решение**

Напишите свое решение ниже

In [ ]:

def sub_renewal_by_day(file_path):

    ...

sub_renewal_by_day('auto_purchase.log')

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt

import pandas as pd

user_answer = pd.read_csv('weekdays.txt')
correct_answer = pd.read_csv('cor_weekdays.txt')

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!
